In [12]:
import os
import pandas as pd
import geopandas as gpd
import rioxarray as riox

import rasterio

import pystac_client
import planetary_computer as pc

import lidar_sampling_functions as lsf

## Downloading data through California Forest Observatory api

In [3]:
#pip install cfo

In [2]:
import cfo

In [4]:
forest_api = cfo.api()
forest_api.authenticate()

CFO E-mail:  c_galazgarcia@ucsb.edu
CFO Password:  ············


200

In [8]:
year = 2018
ca_canopy_itemid = forest_api.search(geography='California', metric='CanopyHeight', year=year)[0]
ca_canopy_itemid

'California-Vegetation-CanopyHeight-2018-Summer-00010m'

In [11]:
ca_canopy_fp = os.path.join(os.getcwd(), 'ca_canopyheight_'+str(year)+'.tif')
ca_canopy_fp

'/home/jovyan/msai4earth-esa/iceplant_detection/model_with_lidar/ca_canopyheight_2018.tif'

In [6]:
# itemid = forest_api.search(geography='SantaBarbaraCounty', metric='CanopyHeight')[0]
# itemid

In [10]:
forest_api.download(ca_canopy_itemid, ca_canopy_fp)

## Clip CA canopy height to SB County

In [13]:
ca_canopy = riox.open_rasterio(ca_canopy_fp, masked =True)
ca_canopy

<xarray.DataArray (band: 1, y: 103969, x: 94338)>
[9808227522 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 3.743e+05 3.743e+05 ... 1.318e+06 1.318e+06
  * y            (y) float64 4.654e+06 4.654e+06 ... 3.614e+06 3.614e+06
    spatial_ref  int64 0
Attributes:
    STATISTICS_MAXIMUM:        80
    STATISTICS_MEAN:           4.3097393919634
    STATISTICS_MINIMUM:        0
    STATISTICS_STDDEV:         7.4818461226433
    STATISTICS_VALID_PERCENT:  42.08
    scale_factor:              1.0
    add_offset:                0.0

In [15]:
sb_fp = '/home/jovyan/msai4earth-esa/shapefiles_n_rasters/SB_geometry/SB_only.shp'
sb = gpd.read_file(sb_fp)
sb

,NAME_PCASE,NAME_UCASE,FMNAME_PC,FMNAME_UC,ABBREV,NUM,ABCODE,FIPS,ANSI,ISLAND,Shape_Leng,Shape_Area,geometry
0,Santa Barbara,SANTA BARBARA,Santa Barbara County,SANTA BARBARA COUNTY,SBA,42,c042,083,083,N,424655.163272,6.608866e+09,"POLYGON ((-7248.153 -322621.781, -7240.234 -32..."
1,Santa Barbara,SANTA BARBARA,Santa Barbara County,SANTA BARBARA COUNTY,SBA,42,c042,083,083,Y,220.962192,3.665768e+03,"POLYGON ((-60843.500 -346204.000, -60854.609 -..."
2,Santa Barbara,SANTA BARBARA,Santa Barbara County,SANTA BARBARA COUNTY,SBA,42,c042,083,083,Y,559.332846,8.479405e+03,"POLYGON ((-58512.125 -362150.188, -58526.457 -..."
3,Santa Barbara,SANTA BARBARA,Santa Barbara County,SANTA BARBARA COUNTY,SBA,42,c042,083,083,Y,859.536811,2.466295e+04,"POLYGON ((28638.146 -401237.438, 28642.877 -40..."
4,Santa Barbara,SANTA BARBARA,Santa Barbara County,SANTA BARBARA COUNTY,SBA,42,c042,083,083,Y,120137.609955,2.502277e+08,"POLYGON ((7485.788 -439053.938, 7488.938 -4389..."
5,Santa Barbara,SANTA BARBARA,Santa Barbara County,SANTA BARBARA COUNTY,SBA,42,c042,083,083,Y,44315.149119,3.852854e+07,"POLYGON ((-39139.164 -440591.344, -39120.691 -..."
6,Santa Barbara,SANTA BARBARA,Santa Barbara County,SANTA BARBARA COUNTY,SBA,42,c042,083,083,Y,1736.785102,1.533134e+05,"POLYGON ((-30672.670 -439903.344, -30699.912 -..."
7,Santa Barbara,SANTA BARBARA,Santa Barbara County,SANTA BARBARA COUNTY,SBA,42,c042,083,083,Y,824.624731,1.531302e+04,"POLYGON ((-38500.484 -439899.375, -38510.887 -..."
8,Santa Barbara,SANTA BARBARA,Santa Barbara County,SANTA BARBARA COUNTY,SBA,42,c042,083,083,Y,584.075619,1.794771e+04,"POLYGON ((-40617.293 -439921.594, -40624.473 -..."
9,Santa Barbara,SANTA BARBARA,Santa Barbara County,SANTA BARBARA COUNTY,SBA,42,c042,083,083,Y,888.098963,4.262273e+04,"POLYGON ((-40294.981 -440075.188, -40312.644 -..."


In [25]:
sb = sb.to_crs(ca_canopy.rio.crs)

In [ ]:
sb_canopy = ca_canopy.rio.clip(sb.geometry, sb.crs)

## Open lidar and create min,max,avg,diff lidar rasters

In [9]:
lidar_year = 2020

In [10]:
fp = os.path.join(os.getcwd(),'SantaBarbaraCounty_lidar_'+str(lidar_year)+'.tif')
lidar_rast_r = rasterio.open(fp)

In [11]:
lsf.save_min_max_rasters(rast_reader = lidar_rast_r, 
                              folder_path = os.path.join(os.getcwd(),'SantaBarbaraCounty_lidar_'+str(lidar_year)),
                              aoi = 'SantaBarbaraCounty',
                              year = 2020)

lsf.save_avg_rasters(rast_reader = lidar_rast_r, 
                              folder_path = os.path.join(os.getcwd(),'SantaBarbaraCounty_lidar_'+str(lidar_year)),
                              aoi = 'SantaBarbaraCounty',
                              year = 2020)

## Open points from given year and aoi

In [12]:
aoi = 'campus_lagoon'
aoi_year = 2020

pts_fp = os.path.join('/home/jovyan/msai4earth-esa/iceplant_detection/data_sampling/points_from_naip_images/',
                     aoi+'_points',
                     aoi+'_points_'+str(aoi_year)+'.csv')

## Obtain CRS from itemid and create geodataframe from points

In [13]:
itemid = pd.read_csv(pts_fp).naip_id[0]
itemid

'ca_m_3411934_sw_11_060_20200521'

In [14]:
# # accesing Azure storage using pystac client
# catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

# # search for naip scene where the pts were sampled from
# search = catalog.search(
#     collections=["naip"],
#     ids = itemid
# )
# item = list(search.get_items())[0]
# epsg_code = item.properties['proj:epsg']

In [15]:
pts = lsf.geodataframe_from_csv(pts_fp, lsf.crs_from_itemid(itemid))
pts.head(3)

/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,iceplant,r,g,b,nir,year,month,day,naip_id,polygon_id,geometry
0,1,78,88,69,161,2020,5,21,ca_m_3411934_sw_11_060_20200521,0,POINT (238491.487 3810779.673)
1,1,85,89,70,162,2020,5,21,ca_m_3411934_sw_11_060_20200521,0,POINT (238490.963 3810789.330)
2,1,74,81,70,144,2020,5,21,ca_m_3411934_sw_11_060_20200521,0,POINT (238484.510 3810781.595)


## Sample LIDAR

In [16]:
pts_xy = lsf.pts_for_lidar_sampling(pts, lidar_rast_r.crs)

In [17]:
lidar_samples = lsf.sample_raster(pts_xy, lidar_rast_r)

In [18]:
lidar_fps = []
for tag in ['maxs_', 'mins_', 'avgs_']:
    lidar_fps.append(os.path.join(os.getcwd(),
                                 'SantaBarbaraCounty_lidar_'+str(lidar_year),
                                 'SantaBarbaraCounty_lidar_'+tag+ str(lidar_year)+'.tif'))

In [19]:
#fp = os.path.join(os.getcwd(),'lidar_maxs.tif')
maxs_rast_r = rasterio.open(lidar_fps[0])
max_samples = lsf.sample_raster(pts_xy, maxs_rast_r)

In [20]:
mins_rast_r = rasterio.open(lidar_fps[1])
min_samples = lsf.sample_raster(pts_xy, mins_rast_r)

In [21]:
#fp = os.path.join(os.getcwd(),'lidar_avgs.tif')
avg_rast_r = rasterio.open(lidar_fps[2])
avg_samples = lsf.sample_raster(pts_xy, avg_rast_r)

In [22]:
pts['lidar'] = lidar_samples
pts['max_lidar']= max_samples
pts['min_lidar'] = min_samples
pts['min_max_diff'] = pts.max_lidar - pts.min_lidar
pts['avg_lidar'] = avg_samples


In [23]:
pts.head(3)

,iceplant,r,g,b,nir,year,month,day,naip_id,polygon_id,geometry,lidar,max_lidar,min_lidar,min_max_diff,avg_lidar
0,1,78,88,69,161,2020,5,21,ca_m_3411934_sw_11_060_20200521,0,POINT (238491.487 3810779.673),2,6,0,6,2.555556
1,1,85,89,70,162,2020,5,21,ca_m_3411934_sw_11_060_20200521,0,POINT (238490.963 3810789.330),0,4,0,4,1.111111
2,1,74,81,70,144,2020,5,21,ca_m_3411934_sw_11_060_20200521,0,POINT (238484.510 3810781.595),4,6,0,6,3.111111


In [24]:
# ptslidar_fp = os.path.join('/home/jovyan/msai4earth-esa/iceplant_detection/data_sampling/points_from_naip_images/',
#                      aoi+'_points',
#                      aoi+'_points_lidar'+str(aoi_year)+'.csv')
# pts.to_csv(ptslidar_fp)

In [25]:
for fp in lidar_fps:
    os.remove(fp)